## Integration of Postgres with Pandas
Pandas is a powerful Python Data Library which is used to process as well as analyze the data. It have robust APIs to work with databases.

Here are the steps involved to use Pandas to work with databases like Postgres.
* We need to make sure `pandas`, `psycopg2-binary` as well as `sqlalchemy` installed using `pip`.
* Pandas uses `sqlalchemy` to interact with database tables based on the connection url.
* `sqlalchemy` is the most popular ORM to hide the complexity of connecting to the databases using our applications.
* Here are the examples using Pandas. 
  * Read from file, Process and Write to Postgres Database Table using Pandas. 
  * Read from Postgres Database Table using Pandas for the validation.

In [1]:
!pip install psycopg2-binary

In [2]:
!pip install pandas

In [3]:
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for greenlet: filename=greenlet-2.0.2-cp39-cp39-macosx_10_9_x86_64.whl size=190831 sha256=d51fc44dfbcb1d46172dcbee2ba147f41a5bbbb150e1eae97dd7e6d979e71ed5
  Stored in directory: /Users/mlevydaniel/Library/Caches/pip/wheels/c4/e2/38/932349e5e893e6d464ea70f98f76a8d7b9ba73e62cc9db5579
Successfully built greenlet


In [4]:
import pandas as pd

In [5]:
columns = ['order_id', 'order_date', 'order_customer_id', 'order_status']

In [6]:
import json

In [7]:
schemas = json.load(open('../../data/retail_db/schemas.json'))

In [8]:
sorted(schemas['orders'], key=lambda col: col['column_position'])

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [9]:
columns = [col['column_name'] for col in sorted(schemas['orders'], key=lambda col: col['column_position'])]

In [10]:
columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [11]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols=None,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace: 'bool' = False,
    skiprows=None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values=None,
    keep_default_na: 'bool' = True,
    na_filter: 'bool' = True,
    verbose: 'bool' = False,
    skip_blank_lines: 'bool' = True,
    parse_dates: 'bool | Sequence[Hashable] | None' = None,
    infer_datetime_format: 'bool | lib.NoDefault' = <no_default>,
    keep_date_col: 'bool' =

In [12]:
orders = pd.read_csv('../../data/retail_db/orders/part-00000', names=columns)

In [13]:
orders

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


In [14]:
type(orders)

pandas.core.frame.DataFrame

In [15]:
orders.shape

(68883, 4)

In [16]:
daily_status_count = orders. \
    groupby(['order_date', 'order_status'])['order_id']. \
    agg(order_count='count'). \
    reset_index()

In [17]:
daily_status_count

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [ ]:
daily_status_count.to_sql?

In [20]:
host = '34.170.93.218'
port = 5432
database = 'retail_db'
user = 'martin'
password = 'martin'

In [21]:
conn_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [22]:
daily_status_count.to_sql(
    'daily_status_count',
    conn_uri,
    if_exists='replace',
    index=False
)

203

In [23]:
pd.read_sql?

Signature:
pd.read_sql(
    sql,
    con,
    index_col: 'str | list[str] | None' = None,
    coerce_float: 'bool' = True,
    params=None,
    parse_dates=None,
    columns: 'list[str] | None' = None,
    chunksize: 'int | None' = None,
    dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>,
    dtype: 'DtypeArg | None' = None,
) -> 'DataFrame | Iterator[DataFrame]'
Docstring:
Read SQL query or database table into a DataFrame.

This function is a convenience wrapper around ``read_sql_table`` and
``read_sql_query`` (for backward compatibility). It will delegate
to the specific function depending on the provided input. A SQL query
will be routed to ``read_sql_query``, while a database table name will
be routed to ``read_sql_table``. Note that the delegated function might
have more specific notes about their functionality not listed here.

Parameters
----------
sql : str or SQLAlchemy Selectable (select or text object)
    SQL query to be executed or a table name.
con : SQLAlch

In [27]:
dsc = pd.read_sql(
    'daily_status_count',
    conn_uri
)

In [29]:
dsc.shape

(3203, 3)

In [30]:
dsc

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [31]:
dsc = pd.read_sql(
    '''
        SELECT 
            order_status
            , SUM(order_count) AS order_count 
        FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri
)

In [32]:
dsc

,order_status,order_count
0,COMPLETE,22899.0
1,PENDING_PAYMENT,15030.0
2,PROCESSING,8275.0
3,PENDING,7610.0
4,CLOSED,7556.0
5,ON_HOLD,3798.0
6,SUSPECTED_FRAUD,1558.0
7,CANCELED,1428.0
8,PAYMENT_REVIEW,729.0
